In [1]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf

In [2]:
from src.data import data

In [3]:
from datetime import datetime as dt

In [4]:
import os
import numpy as np
from IPython import display
from PIL import Image
from glob import glob

In [5]:
def get_filenames_and_labels(data_dir):
    filenames = []
    labels = []
    for d in os.listdir(data_dir):
        category_filenames = glob(f'{data_dir}/{d}/*')
        filenames.extend(category_filenames)
        labels.extend([d for x in range(len(category_filenames))])

    mapping = {l: idx for idx, l in enumerate(np.unique(labels))}

    category = [mapping[l] for l in labels]

    category = tf.keras.utils.to_categorical(category, len(np.unique(labels)))
    
    return filenames, category

In [6]:
import random

In [7]:
def create_tf_dataset(filenames, labels, shuffle_buffer_size=10000, n_repeat_times=100, batch_size=128, shuffle=True):
    def _parse_function(filename, label):
        image_string = tf.read_file(filename)
        image_decoded = tf.image.decode_jpeg(image_string, channels=3)
        image = tf.cast(image_decoded, tf.float32)
        # resized_images = tf.image.resize_images(images, (img_size, img_size))
        resized_image = tf.image.resize_images(image, (img_size, img_size))
        # resized_image = tf.reshape(
        #     resized_image,
        #     (img_size * img_size * 3, ))
        return resized_image/255, label
    
    len_filenames = len(filenames)
    
    tuples = [(f, l) for f, l in zip(filenames, labels)]
    random.seed(14)
    random.shuffle(tuples)
    
    filenames = [t[0] for t in tuples]
    labels = np.array([t[1] for t in tuples])
    
    # step 1
    filenames = tf.constant(filenames)
    labels = tf.constant(labels)

    # step 2: create a dataset returning slices of `filenames`
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

    # step 3: parse every image in the dataset using `map`
    dataset = dataset.map(_parse_function)
    if shuffle:
        dataset = dataset.shuffle(shuffle_buffer_size,
                                  reshuffle_each_iteration=True)
    dataset = dataset.repeat(n_repeat_times).batch(batch_size if shuffle else 1024)

    # step 4: create iterator and final input tensor
    iterator = tf.compat.v1.data.make_one_shot_iterator(dataset)
    images, labels = iterator.get_next()
    return images, labels

In [8]:
""" Convolutional Neural Network.
Build and train a convolutional neural network with TensorFlow.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
"""

# Training Parameters
num_epochs = 100
batch_size = 64
num_classes = 6 
img_size = 128
channels = 3

learning_rate = 0.001
display_step = 50

num_input = (img_size * img_size * channels) 
dropout = 0.5 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, img_size, img_size, channels], name='X')
Y = tf.placeholder(tf.float32, [None, num_classes], name='y')
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [9]:

# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

# Create model
def conv_net(x, weights, biases, dropout):
    # x = tf.reshape(x, shape=[-1, 224, 224, 3]) 
    
    # Convolution Layer
    conv1 = conv2d(x, weights['block_1_wc1'], biases['block_1_bc1'])
    conv1 = tf.compat.v1.layers.batch_normalization(conv1, 
                                                    training=True if dropout != 1 else False)
    conv1 = conv2d(conv1, weights['block_1_wc2'], biases['block_1_bc2'])
    conv1 = tf.compat.v1.layers.batch_normalization(conv1,
                                                    training=True if dropout != 1 else False)
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)
    # Apply Dropout
    conv1 = tf.nn.dropout(conv1, 1 if dropout == 1 else 0.75)
    
    # Convolution Layer
    conv2 = conv2d(conv1, weights['block_2_wc1'], biases['block_2_bc1'])
    conv2 = tf.compat.v1.layers.batch_normalization(conv2, 
                                                    training=True if dropout != 1 else False)
    conv2 = conv2d(conv2, weights['block_2_wc2'], biases['block_2_bc2'])
    conv2 = tf.compat.v1.layers.batch_normalization(conv2,
                                                    training=True if dropout != 1 else False)
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)
    # Apply Dropout
    conv2 = tf.nn.dropout(conv2, 1 if dropout == 1 else 0.75)
        
    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    flatten = tf.contrib.layers.flatten(conv2)
    fc1 = tf.contrib.layers.fully_connected(flatten, 512, )

    # fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    # fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    # fc1 = tf.nn.relu(fc1)
    
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out


# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'block_1_wc1': tf.Variable(tf.random_normal([3, 3, 3, 32])),
    'block_1_wc2': tf.Variable(tf.random_normal([3, 3, 32, 32])),
    
    'block_2_wc1': tf.Variable(tf.random_normal([3, 3, 32, 64])),
    'block_2_wc2': tf.Variable(tf.random_normal([3, 3, 64, 64])),
    
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 512])),
    # 1024 inputs, 6 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([512, num_classes]))
}

biases = {
    'block_1_bc1': tf.Variable(tf.random_normal([32])),
    'block_1_bc2': tf.Variable(tf.random_normal([32])),
    'block_2_bc1': tf.Variable(tf.random_normal([64])),
    'block_2_bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([512])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

### Data

In [10]:
path = data.PATH()
dataset = 'vision_based_dataset'
base_dir = f'{path.PROCESSED_DATA_PATH}/{dataset}'
train_dir = f'{base_dir}/train/'
validation_dir = f'{base_dir}/validation/'
test_dir = f'{base_dir}/test/'

filenames_train, labels_train = get_filenames_and_labels(train_dir)
filenames_valid, labels_valid = get_filenames_and_labels(validation_dir)

train_images, train_labels = create_tf_dataset(filenames_train, labels_train, batch_size=batch_size)

validation_images, validation_labels = create_tf_dataset(filenames_valid, labels_valid, batch_size=batch_size, shuffle=False)

### Model

In [11]:
num_steps = int(num_epochs * len(filenames_train) / batch_size) - 1

In [12]:
# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

saver = tf.train.Saver(max_to_keep=4, keep_checkpoint_every_n_hours=2)

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    start = dt.now()
    
    for step in range(1, num_steps+1):
        #batch_x, batch_y = mnist.train.next_batch(batch_size)
        # batch_x, batch_y = iterator.get_next()
        batch_x, batch_y = sess.run([train_images, train_labels])
        
        batch_start = dt.now()
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.5})
        
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy],
                             feed_dict={
                                 X: batch_x,
                                 Y: batch_y,
                                 keep_prob: 1.0}
                            )
            validation_batch_x, validation_batch_y = sess.run(
                [validation_images, validation_labels])
            
            val_loss, val_acc = sess.run([loss_op, accuracy],
                                            feed_dict={
                                                X: validation_batch_x,
                                                Y: validation_batch_y,
                                                keep_prob: 1.0
                                            })
            end_display = (dt.now() - batch_start).total_seconds() / 60
            print(f"Step {step}/{num_steps} ({step*batch_size} imgs) >> minibatch loss: {np.round(loss, 2)}; acc {np.round(acc, 2)}; val_loss: {np.round(val_loss, 2)}; val_acc {np.round(val_acc, 2)} (Total mins: {np.round((dt.now() - start).total_seconds() / 60, 2)})")

    saved_path = saver.save(sess, './tf_api_model', global_step=step)

    print("Optimization Finished!")

W1201 14:46:34.478761 140145065584448 deprecation.py:323] From <ipython-input-9-ef1fea106582>:21: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
W1201 14:46:34.591379 140145065584448 deprecation.py:506] From <ipython-input-9-ef1fea106582>:28: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1201 14:46:35.733103 140145065584448 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfc

Step 1/178688 (64 imgs) >> minibatch loss: 1100.3299560546875; acc 0.4699999988079071; val_loss: 1810.02001953125; val_acc 0.27000001072883606 (Total mins: 0.59)
Step 50/178688 (3200 imgs) >> minibatch loss: 1.9600000381469727; acc 0.2199999988079071; val_loss: 2.6600000858306885; val_acc 0.2199999988079071 (Total mins: 0.84)
Step 100/178688 (6400 imgs) >> minibatch loss: 2.0; acc 0.20000000298023224; val_loss: 2.109999895095825; val_acc 0.1899999976158142 (Total mins: 1.1)
Step 150/178688 (9600 imgs) >> minibatch loss: 2.049999952316284; acc 0.27000001072883606; val_loss: 2.049999952316284; val_acc 0.18000000715255737 (Total mins: 1.34)
Step 200/178688 (12800 imgs) >> minibatch loss: 1.809999942779541; acc 0.25; val_loss: 2.0; val_acc 0.1899999976158142 (Total mins: 1.6)
Step 250/178688 (16000 imgs) >> minibatch loss: 2.0899999141693115; acc 0.20000000298023224; val_loss: 1.9600000381469727; val_acc 0.1899999976158142 (Total mins: 1.85)
Step 300/178688 (19200 imgs) >> minibatch loss: 

Step 2550/178688 (163200 imgs) >> minibatch loss: 1.7899999618530273; acc 0.27000001072883606; val_loss: 1.7400000095367432; val_acc 0.20000000298023224 (Total mins: 13.42)
Step 2600/178688 (166400 imgs) >> minibatch loss: 1.7799999713897705; acc 0.2800000011920929; val_loss: 1.7400000095367432; val_acc 0.20000000298023224 (Total mins: 13.69)
Step 2650/178688 (169600 imgs) >> minibatch loss: 1.75; acc 0.23000000417232513; val_loss: 1.7599999904632568; val_acc 0.1899999976158142 (Total mins: 13.94)
Step 2700/178688 (172800 imgs) >> minibatch loss: 1.809999942779541; acc 0.25; val_loss: 1.75; val_acc 0.20000000298023224 (Total mins: 14.21)
Step 2750/178688 (176000 imgs) >> minibatch loss: 1.7200000286102295; acc 0.2199999988079071; val_loss: 1.7599999904632568; val_acc 0.20000000298023224 (Total mins: 14.46)
Step 2800/178688 (179200 imgs) >> minibatch loss: 1.7100000381469727; acc 0.25; val_loss: 1.75; val_acc 0.1899999976158142 (Total mins: 14.71)
Step 2850/178688 (182400 imgs) >> minib

Step 5100/178688 (326400 imgs) >> minibatch loss: 1.7599999904632568; acc 0.2199999988079071; val_loss: 1.75; val_acc 0.20999999344348907 (Total mins: 26.54)
Step 5150/178688 (329600 imgs) >> minibatch loss: 1.75; acc 0.2800000011920929; val_loss: 1.7599999904632568; val_acc 0.20000000298023224 (Total mins: 26.83)
Step 5200/178688 (332800 imgs) >> minibatch loss: 1.7200000286102295; acc 0.20000000298023224; val_loss: 1.7599999904632568; val_acc 0.20000000298023224 (Total mins: 27.11)
Step 5250/178688 (336000 imgs) >> minibatch loss: 1.7999999523162842; acc 0.10999999940395355; val_loss: 1.75; val_acc 0.1899999976158142 (Total mins: 27.43)
Step 5300/178688 (339200 imgs) >> minibatch loss: 1.7300000190734863; acc 0.20000000298023224; val_loss: 1.75; val_acc 0.20000000298023224 (Total mins: 27.7)
Step 5350/178688 (342400 imgs) >> minibatch loss: 1.7599999904632568; acc 0.11999999731779099; val_loss: 1.75; val_acc 0.1899999976158142 (Total mins: 27.96)
Step 5400/178688 (345600 imgs) >> min

E1201 15:21:29.108173 140145065584448 ultratb.py:155] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ifranco/Documents/facultad/tesis/tesis_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-01f016bd9dd3>", line 34, in <module>
    sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.5})
  File "/home/ifranco/Documents/facultad/tesis/tesis_env/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 950, in run
    run_metadata_ptr)
  File "/home/ifranco/Documents/facultad/tesis/tesis_env/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1173, in _run
    feed_dict_tensor, options, run_metadata)
  File "/home/ifranco/Documents/facultad/tesis/tesis_env/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1350, in _do_run
    run_metadata)
  File "/home/ifranco/Documents/facultad/tesis/tesis_env/lib/python3.6/site-packages/tensorflow/python/client/session.

KeyboardInterrupt: 

In [ ]:

# Calculate accuracy for 256 MNIST test images
print("Testing Accuracy:", \
    sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                  Y: mnist.test.labels[:256],
                                  keep_prob: 1.0}))